In [1]:
import requests
import pandas as pd
import time
import datetime
from datetime import timedelta
import threading
import logging
from pymongo import MongoClient
from pycaret.regression import *
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
import os

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
API_KEY = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq' 
CURRENCY_PAIRS = ['EUR/USD', 'GBP/CHF', 'USD/CAD', 'EUR/CHF', 'EUR/CAD', 'GBP/EUR', 'GBP/USD', 'GBP/CAD', 'USD/CHF', 'BTC/USD', 'USD/JPY']
CORRELATION_PAIRS = ['BTC/USD','EUR/USD']

In [4]:
hour_count = 0

In [5]:
import sqlite3

# Create the first database and table
fx_conn = sqlite3.connect('fx_data_multiple.db', timeout=40)
fx_cursor = fx_conn.cursor()

fx_cursor.execute('''DROP TABLE IF EXISTS fx_rates''')
fx_cursor.execute('''CREATE TABLE IF NOT EXISTS fx_rates (
                    id INTEGER PRIMARY KEY,
                    currency_pair TEXT,
                    data_timestamp TEXT,
                    db_timestamp TIMESTAMP,
                    max_value REAL,
                    min_value REAL,
                    mean_value REAL,
                    vol REAL,
                    fd REAL
                    )''')

fx_conn.commit()
fx_conn.close()

final_conn = sqlite3.connect('final_fx_data_multiple.db')
final_cursor = final_conn.cursor()
final_cursor.execute('''DROP TABLE IF EXISTS final_fx_rates''')
final_cursor.execute('''CREATE TABLE IF NOT EXISTS final_fx_rates (
                        id INTEGER PRIMARY KEY,
                        currency_pair TEXT,
                        data_timestamp TEXT,
                        db_timestamp TIMESTAMP,
                        max_value REAL,
                        min_value REAL,
                        mean_value REAL,
                        vol REAL,
                        fd REAL,
                        hour INTEGER
                        )''')


final_conn.commit()
final_conn.close()

In [6]:
client = MongoClient('mongodb://localhost:27017/')
mongo_db = client['fx_data_multiple']
fx_rates_mongo = mongo_db['fx_rates_a']

final_mongo_client = MongoClient('mongodb://localhost:27017/')
final_mongo_db = final_mongo_client['fx_data_multiple']
final_fx_rates_mongo = final_mongo_db['fx_rates_f']

In [7]:
import requests
import logging

def fetch_fx_data(pair):
    if pair == 'BTC/USD':
        url = f'https://api.polygon.io/v1/last/crypto/{pair}?'
    else:
        url = f'https://api.polygon.io/v1/conversion/{pair}?amount=1&precision=4'
    
    params = {
       
        'apiKey': API_KEY
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.RequestException as e:
        logging.error(f"Error fetching data for {pair}: {str(e)}")
        return None

def clear_sql_table():
    try:
        conn = sqlite3.connect('fx_data_multiple.db')
        cursor = conn.cursor()
        cursor.execute('DELETE FROM fx_rates')  # Delete all rows in the table
        conn.commit()
        conn.close()
        # print("cleared auxiliary SQLite table")
    except Exception as e:
        logging.error(f"Error clearing SQLite table: {e}")
        

def clear_mongo_collection():
    # print("cleared auxiliary mongo table")
    fx_rates_mongo.delete_many({})  # Delete all documents in the collection

In [8]:
def store_in_final_sql_db(max_value, min_value, mean_value, vol, fd, timestamp, currency_pair, hour):
    try:
        conn = sqlite3.connect('final_fx_data_multiple.db')
        cursor = conn.cursor()

        # Insert the data vector into the final SQLite database
        insert_query = '''INSERT INTO final_fx_rates (currency_pair, data_timestamp, db_timestamp, max_value, min_value, mean_value, vol, fd, hour) 
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)'''
        data_tuple = (currency_pair, timestamp, datetime.datetime.now(), max_value, min_value, mean_value, vol, fd, hour)
        cursor.execute(insert_query, data_tuple)

        conn.commit()  # Commit the transaction
        conn.close()
        # print("Stored data in final SQLite DB")
    except Exception as e:
        logging.error(f"Error storing in final SQLite DB: {e}")

In [9]:
def store_in_final_mongo_db(max_value, min_value, mean_value, vol, fd, timestamp, currency_pair, hour):
    global hour_count
    try:
        data_to_insert = {
            'currency_pair': currency_pair,
            'data_timestamp': timestamp,
            'db_timestamp':datetime.datetime.now(),
            'max_value': max_value,
            'min_value': min_value,
            'mean_value':mean_value,
            'vol':vol,
            'fd':fd,
            'hour': hour
        }
        final_fx_rates_mongo.insert_one(data_to_insert)
        # print("Stored data in final mongo db")
    except Exception as e:
        logging.error(f"Write failed: {e}, retrying...")

In [10]:
def store_in_auxiliary_db_sql(max_value, min_value, mean_value, vol, fd, timestamp, currency_pair):
    try:
        conn = sqlite3.connect('fx_data_multiple.db')
        cursor = conn.cursor()

        # Insert the data vector into the auxiliary SQLite database
        insert_query = '''INSERT INTO fx_rates (currency_pair, data_timestamp, db_timestamp, max_value, min_value, mean_value, vol, fd) 
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?)'''
        data_tuple = (currency_pair, timestamp, datetime.datetime.now(), max_value, min_value, mean_value, vol, fd)
        cursor.execute(insert_query, data_tuple)

        conn.commit()  # Commit the transaction
        conn.close()
        # print("Stored data in auxiliary SQLite DB")
    except Exception as e:
        logging.error(f"Write failed: {e}, retrying...")

In [11]:
def store_in_auxiliary_db_mongo(max_value, min_value, mean_value, vol, fd, timestamp, currency_pair):
    try:
        data_to_insert = {
            'currency_pair': currency_pair,
            'data_timestamp': timestamp,
            'db_timestamp':datetime.datetime.now(),
            'max_value': max_value,
            'min_value': min_value,
            'mean_value':mean_value,
            'vol':vol,
            'fd':fd
        }
        fx_rates_mongo.insert_one(data_to_insert)
        # print("Stored data in auxiliary mongo db")
    except Exception as e:
        logging.error(f"Write failed: {e}, retrying...")

In [12]:
import sqlite3
import logging
import pandas as pd

def return_df_from_final_sql_db():
    try:
        conn = sqlite3.connect('final_fx_data_multiple.db')
        df = pd.read_sql_query('SELECT * FROM final_fx_rates', conn)
        conn.close()
        return df
    except Exception as e:
        logging.error(f"Error reading from final SQLite DB: {e}")

def return_df_from_final_mongo_db():
    try:
        df = pd.DataFrame(list(final_fx_rates_mongo.find()))
        return df
    except Exception as e:
        logging.error(f"Error reading from final MongoDB: {e}")

In [13]:
hourly_means = {pair: [] for pair in CURRENCY_PAIRS}

In [14]:
def correlation(data1, data2):
    print("correlation data1",data1, "data2",data2)
    return pd.Series(data1).corr(pd.Series(data2))

In [15]:
CLASSIFY_CURRENCY_PAIRS = ['EUR/USD', 'EUR/CAD', 'GBP/EUR', 'GBP/CHF', 'USD/CHF', 'USD/CAD']

In [16]:
coerr_mongo_client = MongoClient('mongodb://localhost:27017/')
coerr_mongo_db = coerr_mongo_client['fx_data_multiple']
collection_coerr = coerr_mongo_db['fx_rates_coerr']

In [17]:
def store_correlation_data(correlations):
    try:
        for pair, correlation in correlations.items():
            data_to_insert = {
                'currency_pair_1': pair[0],
                'currency_pair_2': pair[1],
                'correlation': correlation
            }
            collection_coerr.insert_one(data_to_insert)
        print("Stored data in MongoDB successfully.")
    except Exception as e:
        print(f"Error: {e}")

In [18]:
final_df = pd.DataFrame(columns=['currency_pair', 'corr1', 'corr2'])

In [19]:
import pandas as pd
from pymongo import MongoClient
from scipy.stats import pearsonr

def compute_correlations():
    global final_df

    # Load existing data from MongoDB
    existing_df = return_df_from_final_mongo_db()

    # Drop unnecessary columns
    existing_df.drop(columns=['_id', 'db_timestamp','data_timestamp'], inplace=True)

    # Perform correlation computations
    corr_currency = ['EUR/USD', 'BTC/USD']
    pairs = CURRENCY_PAIRS
    correlations = {}
    for pair in pairs:
        for c_pair in corr_currency:
            pair1_mean = existing_df[existing_df['currency_pair'] == pair]['mean_value']
            pair2_mean = existing_df[existing_df['currency_pair'] == c_pair]['mean_value']

            # Check if both datasets have at least two data points
            if len(pair1_mean) >= 2 and len(pair2_mean) >= 2:
                min_len = min(len(pair1_mean), len(pair2_mean))
                pair1_mean = pair1_mean.iloc[:min_len]
                pair2_mean = pair2_mean.iloc[:min_len]

                corr, _ = pearsonr(pair1_mean, pair2_mean)
                if pair not in correlations:
                    correlations[pair] = {}
                correlations[pair][c_pair] = corr

    print("correlations: ", correlations)

    # Update existing data in MongoDB with correlation values
    final_mongo_client = MongoClient('mongodb://localhost:27017/')
    final_mongo_db = final_mongo_client['fx_data_multiple']
    final_fx_rates_mongo = final_mongo_db['fx_rates_f']
    for document in final_fx_rates_mongo.find({}):
        currency_pair = document['currency_pair']
        # Check if correlations are available for the current currency pair
        if currency_pair in correlations:
            document['corr1'] = correlations[currency_pair].get('EUR/USD', None)
            document['corr2'] = correlations[currency_pair].get('BTC/USD', None)
        # Replace the existing document in the collection with the updated document
        final_fx_rates_mongo.replace_one({'_id': document['_id']}, document)

    return final_df


In [20]:
minOfBucket = 6 * 60
numOfHours = 3600 * 5

In [21]:
def calculate_fd(N, max_value, min_value):
    return N / (max_value - min_value) if (max_value - min_value) != 0 else 1

In [22]:
import datetime

def process_currency_pair(currency_pair):
    try:
        # Define the start time for the process
        start_time = datetime.datetime.now()
        
        # Define the total runtime for the process (5 minutes)
        total_runtime = datetime.timedelta(hours=5)
        
        # Calculate the end time
        end_time = start_time + total_runtime
        
        # Define the interval for calculating correlations (1 hour)
        correlation_interval = datetime.timedelta(hours=1)

        # Define the interval for calculating metrics (6 seconds)
        metric_interval = datetime.timedelta(seconds=minOfBucket)
        
        while datetime.datetime.now() < end_time:
            logging.info(f"Starting loop for {currency_pair}")
            start_time_loop = datetime.datetime.now()
            end_time_loop = start_time_loop + metric_interval

            max_value = 0
            min_value = float('inf')
            sum_values = 0
            total = 0
            prev_rate = 0
            prev_vol = 1
            N = 0

            while datetime.datetime.now() < end_time_loop:
                data = fetch_fx_data(currency_pair)
                if currency_pair == 'BTC/USD':
                    rate_key = 'price'
                else:
                    rate_key = 'ask'
                if data:
                    current_rate = data['last'][rate_key]

                    min_value = min(min_value, current_rate)
                    max_value = max(max_value, current_rate)

                    sum_values += current_rate
                    total += 1

                    mean_value = sum_values / total if total else 0
                    vol = (max_value - min_value) / mean_value if mean_value else float('inf')
                    if vol == 0:
                        vol = 1

                    # Remove 'i' from here
                    N += (current_rate - prev_rate) / (0.025 * prev_vol)
                    fd = calculate_fd(N, max_value, min_value)

                    prev_rate = current_rate
                    prev_vol = vol
                    store_in_auxiliary_db_sql(max_value, min_value, mean_value, vol, fd, datetime.datetime.now(), currency_pair)
                    store_in_auxiliary_db_mongo(max_value, min_value, mean_value, vol, fd, datetime.datetime.now(), currency_pair)
                    
            store_in_final_sql_db(max_value, min_value, mean_value, vol, fd, start_time_loop, currency_pair, hour_count)
            store_in_final_mongo_db(max_value, min_value, mean_value, vol, fd, start_time_loop, currency_pair, hour_count)

            clear_sql_table()
            clear_mongo_collection()
            
            logging.info(f"Done calculating for {currency_pair} and waiting for others")
            
            # Calculate correlations every hour after the first hour
            if datetime.datetime.now() >= start_time + correlation_interval:
                df = compute_correlations()
                # Reset the start time for correlation calculation
                start_time += correlation_interval
                
    except Exception as e:
        logging.error(f"Error in processing for {currency_pair}: {e}")

In [23]:
threads = [threading.Thread(target=process_currency_pair, args=(pair,)) for pair in CURRENCY_PAIRS]
for thread in threads:
    thread.start()

for thread in threads:
    thread.join()
    
logging.info("All threads have finished processing.")

2024-05-06 21:03:01,137 - INFO - Starting loop for EUR/USD
2024-05-06 21:03:01,138 - INFO - Starting loop for GBP/CHF
2024-05-06 21:03:01,139 - INFO - Starting loop for USD/CAD
2024-05-06 21:03:01,141 - INFO - Starting loop for EUR/CHF
2024-05-06 21:03:01,145 - INFO - Starting loop for EUR/CAD
2024-05-06 21:03:01,146 - INFO - Starting loop for GBP/EUR
2024-05-06 21:03:01,149 - INFO - Starting loop for GBP/USD
2024-05-06 21:03:01,150 - INFO - Starting loop for GBP/CAD
2024-05-06 21:03:01,152 - INFO - Starting loop for USD/CHF
2024-05-06 21:03:01,152 - INFO - Starting loop for BTC/USD
2024-05-06 21:03:01,153 - INFO - Starting loop for USD/JPY
2024-05-06 21:03:41,078 - ERROR - Error fetching data for EUR/CAD: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v1/conversion/EUR/CAD?amount=1&precision=4&apiKey=beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x30c4aeb60>: Failed to resolve 'api.

correlations:  {'EUR/USD': {'EUR/USD': 1.0, 'BTC/USD': -0.717782886967283}, 'GBP/CHF': {'EUR/USD': -0.8188034910770253, 'BTC/USD': 0.8176568662917961}, 'USD/CAD': {'EUR/USD': -0.9841927395799557, 'BTC/USD': 0.789985235934369}, 'EUR/CHF': {'EUR/USD': -0.7095738930429455, 'BTC/USD': 0.7049147492309876}, 'EUR/CAD': {'EUR/USD': 0.438121022726149, 'BTC/USD': 0.09522646606485007}, 'GBP/EUR': {'EUR/USD': -0.7669837334137425, 'BTC/USD': 0.7394273055105729}, 'GBP/USD': {'EUR/USD': 0.9852129423039415, 'BTC/USD': -0.6496799564797359}, 'GBP/CAD': {'EUR/USD': -0.3550457234519336, 'BTC/USD': 0.6900769472029067}, 'USD/CHF': {'EUR/USD': -0.9561200440917305, 'BTC/USD': 0.772724518221228}, 'BTC/USD': {'EUR/USD': -0.717782886967283, 'BTC/USD': 0.9999999999999999}, 'USD/JPY': {'EUR/USD': -0.9608579894031057, 'BTC/USD': 0.8093367612765798}}
correlations:  {'EUR/USD': {'EUR/USD': 1.0, 'BTC/USD': -0.717782886967283}, 'GBP/CHF': {'EUR/USD': -0.8188034910770253, 'BTC/USD': 0.8176568662917961}, 'USD/CAD': {'EUR

2024-05-06 22:03:03,343 - INFO - Starting loop for USD/CAD
2024-05-06 22:03:03,343 - INFO - Starting loop for EUR/CHF
2024-05-06 22:03:03,347 - INFO - Starting loop for EUR/CAD
2024-05-06 22:03:03,349 - INFO - Starting loop for GBP/EUR
2024-05-06 22:03:03,354 - INFO - Starting loop for USD/JPY
2024-05-06 22:03:03,356 - INFO - Starting loop for GBP/CHF
2024-05-06 22:03:03,360 - INFO - Starting loop for USD/CHF
2024-05-06 22:03:03,363 - INFO - Starting loop for GBP/CAD
2024-05-06 22:03:03,365 - INFO - Starting loop for BTC/USD
2024-05-06 22:03:03,377 - INFO - Starting loop for GBP/USD
2024-05-06 22:03:03,393 - INFO - Starting loop for EUR/USD
2024-05-06 22:03:33,260 - ERROR - Error fetching data for GBP/USD: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v1/conversion/GBP/USD?amount=1&precision=4&apiKey=beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x30c4aeaa0>: Failed to resolve 'api.

correlations: correlations:  {'EUR/USD': {'EUR/USD': 0.9999999999999998, 'BTC/USD': -0.5507537861814772}, 'GBP/CHF': {'EUR/USD': -0.7769371615914015, 'BTC/USD': 0.6899138475656934}, 'USD/CAD': {'EUR/USD': -0.9651570866679415, 'BTC/USD': 0.501682173745369}, 'EUR/CHF': {'EUR/USD': -0.7699276241088076, 'BTC/USD': 0.4962320374690835}, 'EUR/CAD': {'EUR/USD': 0.06164909425283232, 'BTC/USD': -0.16378955203457973}, 'GBP/EUR': {'EUR/USD': 0.022185620932794952, 'BTC/USD': 0.531204918757979}, 'GBP/USD': {'EUR/USD': 0.967993938001985, 'BTC/USD': -0.39198394182907526}, 'GBP/CAD': {'EUR/USD': 0.05987587691474206, 'BTC/USD': 0.35349695148956095}, 'USD/CHF': {'EUR/USD': -0.9567491308354374, 'BTC/USD': 0.5567037017373883}, 'BTC/USD': {'EUR/USD': -0.5507537861814772, 'BTC/USD': 1.0}, 'USD/JPY': {'EUR/USD': -0.9420552195410871, 'BTC/USD': 0.5002247795093716}}
 {'EUR/USD': {'EUR/USD': 0.9999999999999998, 'BTC/USD': -0.5507537861814772}, 'GBP/CHF': {'EUR/USD': -0.7769371615914015, 'BTC/USD': 0.689913847565

2024-05-06 23:03:05,645 - INFO - Starting loop for USD/CHF
2024-05-06 23:03:05,648 - INFO - Starting loop for GBP/CAD
2024-05-06 23:03:05,649 - INFO - Starting loop for USD/CAD
2024-05-06 23:03:05,652 - INFO - Starting loop for GBP/EUR
2024-05-06 23:03:05,653 - INFO - Starting loop for GBP/USD
2024-05-06 23:03:05,657 - INFO - Starting loop for EUR/CAD
2024-05-06 23:03:05,668 - INFO - Starting loop for BTC/USD
2024-05-06 23:03:05,675 - INFO - Starting loop for GBP/CHF
2024-05-06 23:03:05,681 - INFO - Starting loop for EUR/USD
2024-05-06 23:03:05,683 - INFO - Starting loop for EUR/CHF
2024-05-06 23:03:05,684 - INFO - Starting loop for USD/JPY
2024-05-06 23:03:33,784 - ERROR - Error fetching data for EUR/CHF: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v1/conversion/EUR/CHF?amount=1&precision=4&apiKey=beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x30c530580>: Failed to resolve 'api.

...

2024-05-07 01:03:26,866 - ERROR - Error fetching data for GBP/USD: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v1/conversion/GBP/USD?amount=1&precision=4&apiKey=beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x30c530070>: Failed to resolve 'api.polygon.io' ([Errno 8] nodename nor servname provided, or not known)"))
2024-05-07 01:09:10,512 - INFO - Done calculating for USD/CHF and waiting for others
2024-05-07 01:09:10,512 - INFO - Starting loop for USD/CHF
2024-05-07 01:09:10,516 - INFO - Done calculating for USD/JPY and waiting for others
2024-05-07 01:09:10,517 - INFO - Done calculating for GBP/CHF and waiting for others
2024-05-07 01:09:10,517 - INFO - Starting loop for USD/JPY
2024-05-07 01:09:10,517 - INFO - Starting loop for GBP/CHF
2024-05-07 01:09:10,542 - INFO - Done calculating for EUR/USD and waiting for others
2024-05-07 01:09:10,543 - INFO - Starting loop for EUR/USD
2

correlations:  {'EUR/USD': {'EUR/USD': 1.0, 'BTC/USD': -0.34872494262216247}, 'GBP/CHF': {'EUR/USD': 0.09266613882672668, 'BTC/USD': 0.34202229232407133}, 'USD/CAD': {'EUR/USD': -0.8134535915627216, 'BTC/USD': 0.08007033101257037}, 'EUR/CHF': {'EUR/USD': -0.31730419235269897, 'BTC/USD': -0.13290775880302036}, 'EUR/CAD': {'EUR/USD': 0.1236857983352821, 'BTC/USD': -0.3363422438515629}, 'GBP/EUR': {'EUR/USD': 0.039006694227945574, 'BTC/USD': 0.6606865424775212}, 'GBP/USD': {'EUR/USD': 0.8290785702719348, 'BTC/USD': 0.10055798634794035}, 'GBP/CAD': {'EUR/USD': 0.09876275334421827, 'BTC/USD': 0.4530049650284255}, 'USD/CHF': {'EUR/USD': -0.7870183678990657, 'BTC/USD': 0.13155190582399587}, 'BTC/USD': {'EUR/USD': -0.34872494262216247, 'BTC/USD': 0.9999999999999998}, 'USD/JPY': {'EUR/USD': -0.5442545669678326, 'BTC/USD': -0.36520179399261277}}
correlations:  {'EUR/USD': {'EUR/USD': 1.0, 'BTC/USD': -0.34872494262216247}, 'GBP/CHF': {'EUR/USD': 0.09266613882672668, 'BTC/USD': 0.34202229232407133

2024-05-07 02:03:12,340 - INFO - Done calculating for EUR/CHF and waiting for others
2024-05-07 02:03:12,453 - INFO - Done calculating for GBP/EUR and waiting for others


correlations:  {'EUR/USD': {'EUR/USD': 1.0, 'BTC/USD': -0.34872494262216247}, 'GBP/CHF': {'EUR/USD': 0.09266613882672668, 'BTC/USD': 0.34202229232407133}, 'USD/CAD': {'EUR/USD': -0.8134535915627216, 'BTC/USD': 0.08007033101257037}, 'EUR/CHF': {'EUR/USD': -0.05722153089914037, 'BTC/USD': -0.1358639643536707}, 'EUR/CAD': {'EUR/USD': 0.1236857983352821, 'BTC/USD': -0.3363422438515629}, 'GBP/EUR': {'EUR/USD': 0.039006694227945574, 'BTC/USD': 0.6606865424775212}, 'GBP/USD': {'EUR/USD': 0.8290785702719348, 'BTC/USD': 0.10055798634794035}, 'GBP/CAD': {'EUR/USD': 0.35477398937001275, 'BTC/USD': 0.33975941690641676}, 'USD/CHF': {'EUR/USD': -0.7870183678990657, 'BTC/USD': 0.13155190582399587}, 'BTC/USD': {'EUR/USD': -0.34872494262216247, 'BTC/USD': 0.9999999999999998}, 'USD/JPY': {'EUR/USD': -0.5442545669678326, 'BTC/USD': -0.36520179399261277}}
correlations:  {'EUR/USD': {'EUR/USD': 1.0, 'BTC/USD': -0.34872494262216247}, 'GBP/CHF': {'EUR/USD': 0.09266613882672668, 'BTC/USD': 0.34202229232407133

2024-05-07 02:03:12,944 - INFO - All threads have finished processing.


In [24]:
import pandas as pd
from pymongo import MongoClient

# Connect to your MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['fx_data_multiple']  # Replace 'your_database_name' with your actual database name
collection = db['fx_rates_f']  # Replace 'your_collection_name' with your actual collection name

# Query data from MongoDB and convert it to a DataFrame
data = list(collection.find())
df = pd.DataFrame(data)

# Drop _id, data_timestamp, and db_timestamp columns
df.drop(['_id', 'data_timestamp', 'db_timestamp'], axis=1, inplace=True)

# Filter data for currency pairs in CLASSIFY_CURRENCY_PAIRS
filtered_df = df[df['currency_pair'].isin(CLASSIFY_CURRENCY_PAIRS)]

In [25]:
# Group the data by currency pair
grouped = filtered_df.groupby('currency_pair')

# Initialize an empty DataFrame to store aggregated results
aggregated_df = pd.DataFrame()

# Iterate over each group and aggregate every 20 entries
chunk_size = 20
for name, group in grouped:
    numeric_data = group.select_dtypes(include='number')  # Select only numeric columns
    num_chunks = len(numeric_data) // chunk_size
    chunks = [numeric_data[i:i + chunk_size] for i in range(0, len(numeric_data), chunk_size)]
    
    # Calculate the mean for each chunk and store in aggregated DataFrame
    chunk_means = pd.concat([chunk.mean().to_frame().T for chunk in chunks], ignore_index=True)
    aggregated_df = pd.concat([aggregated_df, chunk_means], ignore_index=True)

# Add a column for future mean by shifting the mean_value column if it exists
if 'mean_value' in aggregated_df.columns:
    aggregated_df['future_mean'] = aggregated_df['mean_value'].shift(-1)

# Insert the result into the new collection
synthetic_collection = db['synthetic_currency_data']  # Name for the new collection
synthetic_collection.insert_many(aggregated_df.to_dict('records'))

InsertManyResult([ObjectId('6639c420340bae884ea36ea2'), ObjectId('6639c420340bae884ea36ea3'), ObjectId('6639c420340bae884ea36ea4'), ObjectId('6639c420340bae884ea36ea5'), ObjectId('6639c420340bae884ea36ea6'), ObjectId('6639c420340bae884ea36ea7'), ObjectId('6639c420340bae884ea36ea8'), ObjectId('6639c420340bae884ea36ea9'), ObjectId('6639c420340bae884ea36eaa'), ObjectId('6639c420340bae884ea36eab'), ObjectId('6639c420340bae884ea36eac'), ObjectId('6639c420340bae884ea36ead'), ObjectId('6639c420340bae884ea36eae'), ObjectId('6639c420340bae884ea36eaf'), ObjectId('6639c420340bae884ea36eb0'), ObjectId('6639c420340bae884ea36eb1'), ObjectId('6639c420340bae884ea36eb2'), ObjectId('6639c420340bae884ea36eb3')], acknowledged=True)

In [26]:
from pymongo import MongoClient
import pandas as pd
from pycaret.regression import *

# Step 1: Connect to MongoDB and fetch synthetic data
synthetic_client = MongoClient('mongodb://localhost:27017/')
synthetic_mongo_db = synthetic_client['fx_data_multiple']
synthetic_fx_rates_mongo = synthetic_mongo_db['synthetic_currency_data']

synth_data = pd.DataFrame(list(synthetic_fx_rates_mongo.find()))

synth_data.dropna(subset=['future_mean'], inplace=True)

reg_setup = setup(data=synth_data, target='future_mean', session_id=123,verbose=False)

best_model = compare_models(verbose=False)

final_model = finalize_model(best_model)

evaluate_model(best_model)

model_name = 'best_regression_model'
save_model(final_model, model_name)

predictions = predict_model(best_model, data=synth_data, verbose=False)  # Use the tuned model for predictions
metrics = pull()  # Extract model performance metrics
    
mae, mse, rmse, r_squared, rmsle, mape = list(metrics.values[0][1:])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

Transformation Pipeline and Model Successfully Saved


In [27]:
def calculate_forecastability(filtered_df):
    
    filtered_df['future_mean'] = df['mean_value'].shift(-1) 
    filtered_df.dropna(subset=['future_mean'], inplace=True)
    setup(data=filtered_df, target='future_mean', session_id=123,verbose=False)

    best_model = compare_models(verbose=False)
    tuned_model = tune_model(best_model, verbose=False)
    predictions = predict_model(best_model, data=filtered_df, verbose=False)  # Use the tuned model for predictions
    metrics = pull()  # Extract model performance metrics

    mae, mse, rmse, r_squared, rmsle, mape = list(metrics.values[0][1:])
    
    mae_threshold = 0.002
    mse_threshold = 0.0005
    rmse_threshold = 0.003
    r_squared_threshold = 0.8
    rmsle_threshold = 0.02
    mape_threshold = 0.08

    # Determine forecastability based on updated thresholds
    forecastable_conditions = (
        mae <= mae_threshold or
        mse <= mse_threshold or
        (rmse <= rmse_threshold and r_squared >= r_squared_threshold) or
        (rmsle <= rmsle_threshold and mape <= mape_threshold)
    )
    
    non_forecastable_conditions = (
        mae > mae_threshold * 10 or
        mse > mse_threshold * 10 or
        (rmse > rmse_threshold * 10 and r_squared < r_squared_threshold * 0.1) or
        (rmsle > rmsle_threshold * 10 and mape > mape_threshold * 10)
    )

    # Classify forecastability
    if forecastable_conditions:
        return "FORECASTABLE"
    elif non_forecastable_conditions:
        return "NON-FORECASTABLE"
    else:
        return "UNDEFINED"

In [28]:
def extract_currency_data(df, currency_pair):
    
    return df[df['currency_pair'] == currency_pair]

In [29]:
data_usdchf = extract_currency_data(df, 'USD/CHF')
data_usdcad = extract_currency_data(df, 'USD/CAD')

In [30]:
calculate_forecastability(data_usdchf)

'NON-FORECASTABLE'

In [31]:
calculate_forecastability(data_usdcad)

'NON-FORECASTABLE'

In [32]:
def compute_slope(data):
    n = len(data)
    X = np.arange(n)
    y = data.values
    slope = np.sum((X - np.mean(X)) * (y - np.mean(y))) / np.sum((X - np.mean(X)) ** 2)
    return slope

In [33]:
import numpy as np

In [34]:
import pandas as pd

# Function to compute the slope (placeholder example)
def compute_slope(prices):
    return (prices.iloc[-1] - prices.iloc[0]) / len(prices)

# Function to get slope series for a given price series
def get_slope_series(price_series, window=20):
    slopes = []
    for i in range(len(price_series) - window + 1):
        slope = compute_slope(price_series[i:i + window])
        slopes.append(slope)
    
    # Create a Series with the correct index
    slope_index = price_series.index[window - 1:]
    return pd.Series(slopes, index=slope_index)

# Assuming 'data' is the DataFrame with the currency pairs data
def calculate_slope_for_pairs(df, pair, window=20):
    # Filter the DataFrame for the specific currency pair
    filtered_df = df[df['currency_pair'] == pair]
    
    # Apply the slope function to the 'mean_value' column
    return get_slope_series(filtered_df['mean_value'], window)

# Example usage:
# Calculate slopes for GBP/USD and USD/JPY with appropriate window size
gbpusd_slope = calculate_slope_for_pairs(df, 'GBP/USD', window=20)
usdjpy_slope = calculate_slope_for_pairs(df, 'USD/JPY', window=20)

# Add the slopes to the original DataFrame, matching indices
df.loc[gbpusd_slope.index, 'GBPUSD_slope'] = gbpusd_slope
df.loc[usdjpy_slope.index, 'USDJPY_slope'] = usdjpy_slope

In [35]:
import time
import pandas as pd

def calculate_profit_loss(initial_price, final_price, units):
    return (final_price - initial_price) * units

# Main loop
usd_amount = 100  
usd_to_gbp_ratio = 100 

start_hour = pd.Timestamp.now().hour
trade_hours = [5, 6, 7]
closing_hour = 8
trade_results = []

for trade_hour in trade_hours:
    # Fetch initial prices
    gbpusd_data_start = fetch_fx_data('GBP/USD')
    usdjpy_data_start = fetch_fx_data('USD/JPY')

    gbpusd_price_start = gbpusd_data_start['last']['ask']
    usdjpy_price_start = usdjpy_data_start['last']['ask'] / usd_to_gbp_ratio  # Adjust ratio

    # Wait for an hour (3600 seconds)
    time.sleep(3600)

    # Fetch final prices
    gbpusd_data_end = fetch_fx_data('GBP/USD')
    usdjpy_data_end = fetch_fx_data('USD/JPY')

    gbpusd_price_end = gbpusd_data_end['last']['ask']
    usdjpy_price_end = usdjpy_data_end['last']['ask'] / usd_to_gbp_ratio  # Adjust ratio

    # Calculate profit or loss
    gbpusd_pl = calculate_profit_loss(gbpusd_price_start, gbpusd_price_end, usd_amount)
    usdjpy_pl = calculate_profit_loss(usdjpy_price_start, usdjpy_price_end, usd_amount * usd_to_gbp_ratio)

    total_pl = gbpusd_pl + usdjpy_pl
    trade_results.append(total_pl)
    print(f"Trade at hour {trade_hour}: Profit/Loss: ${total_pl}")

# Total Profit/Loss over all trades
total_profit_loss = sum(trade_results)
print(f"Total Profit/Loss over 3-step L/S: ${total_profit_loss}")

Trade at hour 5: Profit/Loss: $-19.348999999998306
Trade at hour 6: Profit/Loss: $-32.55700000000199
Trade at hour 7: Profit/Loss: $38.508000000001296
Total Profit/Loss over 3-step L/S: $-13.397999999999001
